### Country Club SqLite db (local)

In [60]:
import sqlite3

# Connect to a SQLite database (creates a new one if it doesn't exist)
import sqlite3  # Ensure sqlite3 module is imported

# Provide the full file path here if the database is not in the same directory as your Python script
db_name = 'sqlite_db_pythonsqlite.db'

try:
    # Try to connect to the database
    conn = sqlite3.connect(db_name)

except sqlite3.OperationalError as e:
    print(
        f"Unable to connect to the database {db_name}. Please make sure the database exists and you have the correct permissions to access it.")
    print(e)
cursor = conn.cursor()



# Close the connection
conn.close()

In [62]:
import sqlite3

# Connect to the SQLite database
try:
	conn = sqlite3.connect('sqlite_db_pythonsqlite.db')
	cursor = conn.cursor()
except sqlite3.Error as e:
	print(f"Error connecting to database: {e}")
	raise

# Verify tables exist
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]
required_tables = ['Bookings', 'Facilities', 'Members']
missing_tables = [t for t in required_tables if t not in tables]
if missing_tables:
	print(f"Error: Missing tables: {missing_tables}")
	print("Ensure sqlite_db_pythonsqlite.db contains Bookings, Facilities, and Members.")
	conn.close()
	raise ValueError("Missing required tables")

# Q9: Bookings on 2012-09-14 costing more than $30
print("Q9: Bookings on 2012-09-14 costing more than $30")
query_q9 = """
SELECT 
    f.name AS facility_name,
    m.firstname || ' ' || m.surname AS member_name,
    b.cost
FROM (
    SELECT 
        bookid,
        b.facid,  -- Explicitly use b.facid to avoid ambiguity
        memid,
        starttime,
        CASE 
            WHEN memid = 0 THEN slots * f.guestcost
            ELSE slots * f.membercost
        END AS cost
    FROM Bookings b
    JOIN Facilities f ON b.facid = f.facid
) b
JOIN Facilities f ON b.facid = f.facid
JOIN Members m ON b.memid = m.memid
WHERE DATE(b.starttime) = '2012-09-14'
AND b.cost > 30
ORDER BY b.cost DESC;
"""
try:
	cursor.execute(query_q9)
	for row in cursor.fetchall():
		print(row)
except sqlite3.Error as e:
	print(f"Q9 Error: {e}")

# Q10: Facilities with total revenue less than 1000
print("\nQ10: Facilities with total revenue less than 1000")
query_q10 = """
SELECT 
    f.name AS facility_name,
    SUM(CASE 
        WHEN b.memid = 0 THEN b.slots * f.guestcost 
        ELSE b.slots * f.membercost 
    END) AS total_revenue
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
GROUP BY f.name
HAVING total_revenue < 1000
ORDER BY total_revenue;
"""
try:
	cursor.execute(query_q10)
	for row in cursor.fetchall():
		print(row)
except sqlite3.Error as e:
	print(f"Q10 Error: {e}")

# Q11: Members and their recommenders
print("\nQ11: Members and their recommenders")
query_q11 = """
SELECT 
    m1.surname || ', ' || m1.firstname AS member_name,
    CASE 
        WHEN m2.memid IS NULL THEN 'None'
        ELSE m2.surname || ', ' || m2.firstname
    END AS recommender_name
FROM Members m1
LEFT JOIN Members m2 ON m1.recommendedby = m2.memid
WHERE m1.memid != 0
ORDER BY m1.surname, m1.firstname;
"""
try:
	cursor.execute(query_q11)
	for row in cursor.fetchall():
		print(row)
except sqlite3.Error as e:
	print(f"Q11 Error: {e}")

# Q12: Facility usage by members (excluding guests)
print("\nQ12: Facility usage by members (excluding guests)")
query_q12 = """
SELECT 
    f.name,
    SUM(b.slots) AS member_usage
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name
ORDER BY f.name;
"""
try:
	cursor.execute(query_q12)
	for row in cursor.fetchall():
		print(row)
except sqlite3.Error as e:
	print(f"Q12 Error: {e}")

# Q13: Facility usage by month (excluding guests)
print("\nQ13: Facility usage by month (excluding guests)")
query_q13 = """
SELECT 
    f.name,
    strftime('%Y-%m', b.starttime) AS month,
    SUM(b.slots) AS member_usage
FROM Bookings b
JOIN Facilities f ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name, strftime('%Y-%m', b.starttime)
ORDER BY f.name, month;
"""
try:
	cursor.execute(query_q13)
	for row in cursor.fetchall():
		print(row)
except sqlite3.Error as e:
	print(f"Q13 Error: {e}")

# Close the connection
conn.close()

Q9: Bookings on 2012-09-14 costing more than $30
('Massage Room 2', 'GUEST GUEST', 320)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Tennis Court 2', 'GUEST GUEST', 150)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 2', 'GUEST GUEST', 75)
('Squash Court', 'GUEST GUEST', 70.0)
('Massage Room 1', 'Jemima Farrell', 39.6)
('Squash Court', 'GUEST GUEST', 35.0)
('Squash Court', 'GUEST GUEST', 35.0)

Q10: Facilities with total revenue less than 1000
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)

Q11: Members and their recommenders
('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Farrell, David', 'None')
('Farrell, Jemima

Q9: Bookings on 2012-09-14 costing more than $30
('Massage Room 2', 'GUEST GUEST', 320)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Massage Room 1', 'GUEST GUEST', 160)
('Tennis Court 2', 'GUEST GUEST', 150)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 1', 'GUEST GUEST', 75)
('Tennis Court 2', 'GUEST GUEST', 75)
('Squash Court', 'GUEST GUEST', 70.0)
('Massage Room 1', 'Jemima Farrell', 39.6)
('Squash Court', 'GUEST GUEST', 35.0)
('Squash Court', 'GUEST GUEST', 35.0)

Q10: Facilities with total revenue less than 1000
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)

Q11: Members and their recommenders
('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Farrell, David', 'None')
('Farrell, Jemima', 'None')
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rownam, Tim', 'None')
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Darren', 'None')
('Smith, Darren', 'None')
('Smith, Jack', 'Smith, Darren')
('Smith, Tracy', 'None')
('Stibbons, Ponder', 'Tracy, Burton')
('Tracy, Burton', 'None')
('Tupperware, Hyacinth', 'None')
('Worthington-Smyth, Henry', 'Smith, Tracy')

Q12: Facility usage by members (excluding guests)
('Badminton Court', 1086)
('Massage Room 1', 884)
('Massage Room 2', 54)
('Pool Table', 856)
('Snooker Table', 860)
('Squash Court', 418)
('Table Tennis', 794)
('Tennis Court 1', 957)
('Tennis Court 2', 882)

Q13: Facility usage by month (excluding guests)
('Badminton Court', '2012-07', 165)
('Badminton Court', '2012-08', 414)
('Badminton Court', '2012-09', 507)
('Massage Room 1', '2012-07', 166)
('Massage Room 1', '2012-08', 316)
('Massage Room 1', '2012-09', 402)
('Massage Room 2', '2012-07', 8)
('Massage Room 2', '2012-08', 18)
('Massage Room 2', '2012-09', 28)
('Pool Table', '2012-07', 110)
('Pool Table', '2012-08', 303)
('Pool Table', '2012-09', 443)
('Snooker Table', '2012-07', 140)
('Snooker Table', '2012-08', 316)
('Snooker Table', '2012-09', 404)
('Squash Court', '2012-07', 50)
('Squash Court', '2012-08', 184)
('Squash Court', '2012-09', 184)
('Table Tennis', '2012-07', 98)
('Table Tennis', '2012-08', 296)
('Table Tennis', '2012-09', 400)
('Tennis Court 1', '2012-07', 201)
('Tennis Court 1', '2012-08', 339)
('Tennis Court 1', '2012-09', 417)
('Tennis Court 2', '2012-07', 123)
('Tennis Court 2', '2012-08', 345)
('Tennis Court 2', '2012-09', 414)
